In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Introduction**

In [ ]:

features = pd.read_csv("/kaggle/input/walmart-sales-prediction/features.csv")
stores = pd.read_csv("/kaggle/input/walmart-sales-prediction/stores.csv")
train = pd.read_csv("/kaggle/input/walmart-sales-prediction/train.csv")
test = pd.read_csv("/kaggle/input/walmart-sales-prediction/test.csv")

In [ ]:
print("Features shape is {}".format(features.shape))
print("Stores shape is {}".format(stores.shape))
print("Train shape is {}".format(train.shape))
print("Test shape is {}".format(test.shape))

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
features.head(3)

In [ ]:
stores.head(3)

In [ ]:
train_df = train.merge(features,how="left",on=['Store','Date','IsHoliday']).merge(stores,how='left',on='Store')

In [ ]:
print(train_df.head(3))
print(train_df.shape)

In [ ]:
test_df = test.merge(features,how="left",on=['Store','Date','IsHoliday']).merge(stores,how='left',on='Store')

In [ ]:
test_df

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
columns_name = train_df.columns
sum_na = train_df.isna().sum()
d = {"Sum_Na":sum_na,}
df_na = pd.DataFrame(d)
df_na['Percentage of Na'] = df_na['Sum_Na']/421570*100
df_na

In [ ]:
train_df.fillna(0,inplace=True)

In [ ]:
columns_name = train_df.columns
sum_na = test_df.isna().sum()
d = {"Sum_Na":sum_na,}
df_na = pd.DataFrame(d)
df_na['Percentage of Na'] = df_na['Sum_Na']/421570*100
df_na

In [ ]:
test_df.describe().T

In [ ]:
test_df['IsHoliday'] = test_df['IsHoliday'].astype("int")

In [ ]:
values = {'MarkDown1':0,'MarkDown2':0,'MarkDown3':0,'MarkDown4':0,'CPI':176.96,'Unemployment':6.86}
test_df.fillna(value = values,inplace=True)

*Na Values are filled!*

In [ ]:
train_df['IsHoliday'] = train_df['IsHoliday'].astype("int")

In [ ]:
#EDA
# Compute the correlation matrix
corr = train_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 13))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8,
            square=True, linewidths=.5, cbar_kws={"shrink": .2},annot=True)




In [ ]:
plt.figure(figsize=(20,5))
sns.distplot(train_df['Weekly_Sales'], bins=40, kde=True, color='red')
plt.title('Weekly_Sales distribution')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
sns.boxplot(x='Weekly_Sales', y='IsHoliday', data=train_df, orient='h', palette='afmhot_r')  # Effect of IsHoliday on Weekly Sales
plt.show()


In [ ]:
plt.figure(figsize=(20,15))
cols_outlier = train_df[['Weekly_Sales', 'Fuel_Price', 'Size', 'CPI', 'Dept', 'Temperature','MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'Unemployment']]
fig, axes = plt.subplots(4,3,figsize=(18,12))
fig.suptitle('Outliers in the numerical features',fontsize=18, color = '#06917e', x = 0.5, y = 1.05)
index = [(i,j) for i in range(4) for j in range(3)]
index_count=0
for col in cols_outlier.columns:
    sns.boxplot(x=col, ax=axes[index[index_count]], data=train_df, palette='afmhot_r')
    index_count += 1
    plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3,2,figsize=(18,12))
ax_index = [(i,j) for i in range(3) for j in range(2)]
index_number = 0
fig.suptitle('Effect of various factors on Weekly Sales',fontsize=18, color = '#06917e', y = 1.05)
for i in ['Unemployment','IsHoliday','Size','CPI','Temperature','Fuel_Price']:
    sns.scatterplot(x=i, y='Weekly_Sales', data=train_df, ax=axes[ax_index[index_number]])
    index_number += 1
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(20,15))
sns.boxplot(x='Size', y='Type', data=train_df, palette='afmhot_r')
plt.title('Size of the Store with respect to Type')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
#Average Sales per stores
avg_sales_per_store = train_df.groupby(by='Store')['Weekly_Sales'].mean()
sns.barplot(x = avg_sales_per_store.index, y=avg_sales_per_store)
plt.title('Average Sales per Store')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
avg_sales_per_dept = train_df.groupby(by='Dept')['Weekly_Sales'].mean()
sns.barplot(x = avg_sales_per_dept.index, y=avg_sales_per_dept)
plt.title('Average Sales per Department')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# Pipeline


pipeline = Pipeline(steps=[
    ('One_Hot_Encoder',OneHotEncoder(drop='first')),
    ('scaler',StandardScaler(with_mean=False)),
    ('model',GradientBoostingRegressor())
])


In [ ]:

train_df.info()

In [ ]:
test_df = test_df.drop('Date',axis=1)
test_df.info()

In [ ]:
# Train Test Split
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)
X = train_df.drop('Weekly_Sales',axis=1)
y = train_df['Weekly_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
params = {
    'model__learning_rate' : [0.01,0.1],
    'model__max_depth' : [3,5]
}

In [ ]:
random_search = RandomizedSearchCV(pipeline,params,cv=3,n_jobs=-1,verbose = 2)
random_search.fit(X_train,y_train)

In [ ]:
y_pred = random_search.predict(X_test)

In [ ]:
random_search.best_params_

In [ ]:
random_search_new =GradientBoostingRegressor(max_depth=5,learning_rate=0.1)
random_search_new.fit(X_train,y_train)

train_pred = random_search_new.predict(X_train)
test_pred = random_search_new.predict(X_test)

In [ ]:
print("RMSE of train is : {}".format(np.sqrt(mean_squared_error(y_train,train_pred))))
print("RMSE of Test is : {}".format(np.sqrt(mean_squared_error(y_test,test_pred))))

In [ ]:
feature_importance = random_search_new.feature_importances_
feature_imp = pd.DataFrame(sorted(zip(random_search_new.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp[-10:-1].sort_values(by="Value", ascending=False))
plt.title('GBM Features (avg over folds)')
plt.tight_layout()
plt.show()